##### Load Package

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import (AIMessage, HumanMessage, SystemMessage)
from langchain.prompts import HumanMessagePromptTemplate

from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain

from langchain_community.document_loaders import TextLoader

import os
from dotenv import load_dotenv

In [3]:
load_dotenv(dotenv_path = "../Key/.env")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Summarize Short Speech

In [ ]:
chat = ChatOpenAI()

In [3]:
speech = '''
Chinese New Year or the Spring Festival (see also § Names) is a festival that celebrates the beginning of a new year on the traditional lunisolar Chinese calendar. Marking the end of winter and the beginning of spring, observances traditionally take place from Chinese New Year's Eve, the evening preceding the first day of the year, to the Lantern Festival, held on the 15th day of the year. The first day of Chinese New Year begins on the new moon that appears between 21 January and 20 February.[a]

Chinese New Year is one of the most important holidays in Chinese culture. It has influenced similar celebrations in other cultures, commonly referred to collectively as Lunar New Year, such as the Losar of Tibet, the Tết of Vietnam, the Korean New Year, and the Ryukyu New Year.[3][4][5] It is also celebrated worldwide in regions and countries that house significant Overseas Chinese or Sinophone populations, especially in Southeast Asia. These include Singapore,[6] Brunei, Cambodia, Indonesia, Malaysia, Myanmar,[7] the Philippines,[8] Thailand, and Vietnam. It is also prominent beyond Asia, especially in Australia, Canada, France, Mauritius,[9] New Zealand, Peru,[10] South Africa, the United Kingdom, and the United States, as well as in many European countries.[11][12][13]

The Chinese New Year is associated with several myths and customs. The festival was traditionally a time to honour deities as well as ancestors.[14] Within China, regional customs and traditions concerning the celebration of the New Year vary widely,[15] and the evening preceding the New Year's Day is frequently regarded as an occasion for Chinese families to gather for the annual reunion dinner. It is also a tradition for every family to thoroughly clean their house, in order to sweep away any ill fortune and to make way for incoming good luck. Another practiced custom is the decoration of windows and doors with red paper-cuts and couplets. Popular themes among these paper-cuts and couplets include good fortune or happiness, wealth, and longevity. Other activities include lighting firecrackers and giving money in red envelopes.

'''

In [9]:
prompt_template = ChatPromptTemplate.from_messages([
    SystemMessage(content = "you are an expert assistant with expertize in summarizing articles"),
    HumanMessagePromptTemplate.from_template("please provide a short and concise summary of the following speech:\n Text: {input_speech}")
    ]
)

prompt = prompt_template.format_messages(input_speech = speech)

print(prompt)

chat.invoke(prompt)

[SystemMessage(content='you are an expert assistant with expertize in summarizing articles'), HumanMessage(content="please provide a short and concise summary of the following speech:\n Text: \nChinese New Year or the Spring Festival (see also § Names) is a festival that celebrates the beginning of a new year on the traditional lunisolar Chinese calendar. Marking the end of winter and the beginning of spring, observances traditionally take place from Chinese New Year's Eve, the evening preceding the first day of the year, to the Lantern Festival, held on the 15th day of the year. The first day of Chinese New Year begins on the new moon that appears between 21 January and 20 February.[a]\n\nChinese New Year is one of the most important holidays in Chinese culture. It has influenced similar celebrations in other cultures, commonly referred to collectively as Lunar New Year, such as the Losar of Tibet, the Tết of Vietnam, the Korean New Year, and the Ryukyu New Year.[3][4][5] It is also c

AIMessage(content="The Chinese New Year, also known as the Spring Festival, is a significant celebration on the traditional lunisolar Chinese calendar. It marks the transition from winter to spring and is observed from Chinese New Year's Eve to the Lantern Festival. Celebrated globally, it influences similar Lunar New Year celebrations in other cultures. The festival is associated with various customs such as honoring deities and ancestors, family reunions, house cleaning for good luck, decorating with red paper-cuts, lighting firecrackers, and giving money in red envelopes.")

# Summarize An Article

In [19]:
loader = TextLoader("Amazon_Transcript.txt")

document = loader.load()

document

[Document(page_content="Hello, and welcome to our Q4 2016 financial results conference call\nJoining us today to answer your questions is Brian Olsavsky, our CFO\nAs you listen to today's conference call, we encourage you to have our press release in front of you, which includes our financial results as well as metrics and commentary on the quarter\nPlease note, unless otherwise stated, all comparisons in this call will be against our results for the comparable period of 2015. Our comments and responses to your questions reflect management's views as of today, February 2, 2017 only and will include forward-looking statements\nActual results may differ materially\nAdditional information about factors that could potentially impact our financial results is included in today's press release and our filings with the SEC, including our most recent Annual Report on Form 10-K and subsequent filings\nDuring this call, we may discuss certain non-GAAP financial measures\nIn our press release, sli

In [22]:
prompt_template = PromptTemplate(
    template = '''Write a concise and short summary of the following speech.
    Speech: {text}
    ''',
    input_variables = ["text"]
)

llm = ChatOpenAI(model_name = "gpt-3.5-turbo",
                 temperature = 0)

chain = load_summarize_chain(
    llm = llm,
    chain_type = "stuff",
    prompt = prompt_template

)

summary = chain.invoke({"input_documents": document},
                        return_only_outputs = True)

print(summary)

{'output_text': "The speech provides an overview of Amazon's Q4 2016 financial results conference call, with CFO Brian Olsavsky answering questions. It includes information on financial results, forward-looking statements, non-GAAP financial measures, guidance, SEC filings, and updates on Form 10-K. The Q&A session covers topics such as FBA offering, international sellers, unit growth, customer count, AWS customers, and other revenue sources. The call concludes with a reminder of the replay availability on the Investor Relations website."}


# Summarize A Long Article

* divide the huge article into chunks \
  summarize each chunk using LLM \
  combine all summaries and generate a final summary
  

In [ ]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo",
                 temperature = 0)

In [34]:
loader = TextLoader("Amazon_Transcript.txt")

document = loader.load()

document

[Document(page_content="Hello, and welcome to our Q4 2016 financial results conference call\nJoining us today to answer your questions is Brian Olsavsky, our CFO\nAs you listen to today's conference call, we encourage you to have our press release in front of you, which includes our financial results as well as metrics and commentary on the quarter\nPlease note, unless otherwise stated, all comparisons in this call will be against our results for the comparable period of 2015. Our comments and responses to your questions reflect management's views as of today, February 2, 2017 only and will include forward-looking statements\nActual results may differ materially\nAdditional information about factors that could potentially impact our financial results is included in today's press release and our filings with the SEC, including our most recent Annual Report on Form 10-K and subsequent filings\nDuring this call, we may discuss certain non-GAAP financial measures\nIn our press release, sli

In [35]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
                                               chunk_overlap = 200)

chunks = text_splitter.split_documents(document)

print(chunks)
print(len(chunks))

[Document(page_content="Hello, and welcome to our Q4 2016 financial results conference call\nJoining us today to answer your questions is Brian Olsavsky, our CFO\nAs you listen to today's conference call, we encourage you to have our press release in front of you, which includes our financial results as well as metrics and commentary on the quarter\nPlease note, unless otherwise stated, all comparisons in this call will be against our results for the comparable period of 2015. Our comments and responses to your questions reflect management's views as of today, February 2, 2017 only and will include forward-looking statements\nActual results may differ materially\nAdditional information about factors that could potentially impact our financial results is included in today's press release and our filings with the SEC, including our most recent Annual Report on Form 10-K and subsequent filings\nDuring this call, we may discuss certain non-GAAP financial measures", metadata={'source': 'Ama

In [36]:
map_prompt_template = PromptTemplate(
    template = """please summarize the below speech
    Speech: {text}
    """,
    input_variables = ["text"]
)


combine_prompt_template = PromptTemplate(
    template = """
    provide a final summary of the entire speech with these important points.
    add a generic motivation title.
    start the precise summary with and introduction and provide the summary in the number points for the speech
    Speech: {text}
    """,
    input_variables = ["text"]
)


chain = load_summarize_chain(
    llm,
    chain_type = "map_reduce",
    map_prompt = map_prompt_template,
    combine_prompt = combine_prompt_template,
    return_intermediate_steps = True
)

summary = chain.invoke({"input_documents": chunks}, return_only_outputs = True)

summary

{'intermediate_steps': ['The speaker welcomes the audience to the Q4 2016 financial results conference call and introduces Brian Olsavsky, the CFO. They encourage listeners to have the press release with the financial results in front of them. They note that all comparisons will be against the results from 2015 and that their comments are as of February 2, 2017 and may include forward-looking statements. They caution that actual results may differ and provide information on factors that could impact financial results. They also mention that they may discuss non-GAAP financial measures during the call.',
  'The speaker will discuss non-GAAP financial measures during the call and directs listeners to additional disclosures on their website. The guidance provided takes into account current order trends and various factors that could impact results, such as foreign exchange rates and global economic conditions. The guidance also assumes no additional business acquisitions, investments, res